<a href="https://colab.research.google.com/github/morozovsolncev/gravitation/blob/main/Gravitation_OTO_minus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.use('Agg')

class HierarchicalBPPSimulator:
    def __init__(self):
        # Физические константы
        self.G = 6.67430e-11
        self.c = 299792458

        # Массы тел
        sun_mass = 1.989e30
        earth_mass = 5.972e24
        moon_mass = 7.348e22

        # Расстояния
        earth_sun_dist = 1.496e11
        moon_earth_dist = 3.844e8

        # Рассчет барицентра Земля-Луна
        total_mass = earth_mass + moon_mass
        barycenter_offset = moon_mass * moon_earth_dist / total_mass

        # Орбитальная скорость Земли вокруг Солнца
        earth_sun_velocity = np.sqrt(self.G * sun_mass / earth_sun_dist)

        # Орбитальная скорость Луны вокруг барицентра
        orbital_velocity = np.sqrt(self.G * total_mass / moon_earth_dist)

        # Иерархическая система тел
        self.system = {
            "Sun": {
                "name": "Sun",
                "mass": sun_mass,
                "position": np.array([0.0, 0.0]),
                "velocity": np.array([0.0, 0.0]),
                "children": ["Earth"],
                "local_time_factor": 1.0,
                "level": 0
            },
            "Earth": {
                "name": "Earth",
                "mass": earth_mass,
                "position": np.array([earth_sun_dist - barycenter_offset, 0.0]),
                "velocity": np.array([0.0, earth_sun_velocity - orbital_velocity * moon_mass / total_mass]),
                "parent": "Sun",
                "children": ["Moon"],
                "local_time_factor": 1.0,
                "level": 1
            },
            "Moon": {
                "name": "Moon",
                "mass": moon_mass,
                "position": np.array([earth_sun_dist + moon_earth_dist - barycenter_offset, 0.0]),
                "velocity": np.array([0.0, earth_sun_velocity + orbital_velocity * earth_mass / total_mass]),
                "parent": "Earth",
                "local_time_factor": 1.000075,
                "level": 2
            }
        }

        # Параметры симуляции
        self.t_max = 27.3 * 24 * 3600  # 1 лунный месяц
        self.dt = 1800  # Уменьшенный шаг (30 минут)
        self.t_points = np.arange(0, self.t_max, self.dt)

        # Коэффициент калибровки гравитации
        self.grav_coeff = 0.5  # Уменьшен для увеличения орбитального периода

    def get_resonance_strength(self, target, source, r, v_rel):
        """Улучшенный расчет силы резонанса"""
        # Базовый резонанс
        base_res = 2 * self.G * source["mass"] / (self.c**2 * r)

        # Иерархическое усиление
        level_diff = abs(target["level"] - source["level"])

        # Усиление для Солнца
        if source["name"] == "Sun":
            hierarchy_boost = 1.5 * (4 - min(level_diff, 4))
        else:
            hierarchy_boost = 10**(3 - min(level_diff, 3))

        # Релятивистский фактор (замедление времени)
        time_dilation = np.exp(-base_res * hierarchy_boost)

        # Допплеровский фактор (влияние относительной скорости)
        r_vec = source["position"] - target["position"]
        vr = np.dot(v_rel, r_vec / r) if r > 0 else 0
        doppler_factor = 1 + vr / self.c

        return np.exp(base_res * hierarchy_boost) * time_dilation * doppler_factor

    def acceleration(self, body_name):
        body = self.system[body_name]
        a_total = np.zeros(2)

        # Центробежное ускорение (для вращающихся систем)
        if "parent" in body:
            parent = self.system[body["parent"]]
            r_vec = body["position"] - parent["position"]
            r = np.linalg.norm(r_vec)
            if r > 0:
                v_tangent = np.array([-body["velocity"][1], body["velocity"][0]])
                centrifugal = np.dot(body["velocity"], v_tangent) / r**2 * r_vec
                a_total += centrifugal

        for source_name, source in self.system.items():
            if source_name == body_name:
                continue

            r_vec = source["position"] - body["position"]
            r = np.linalg.norm(r_vec)
            if r < 1e6:
                continue

            v_rel = source["velocity"] - body["velocity"]
            direction = r_vec / r

            p_res = self.get_resonance_strength(body, source, r, v_rel)

            # Основной градиент
            dp_dr = p_res * (2 * self.G * source["mass"]) / (self.c**2 * r**2)

            # Релятивистская поправка (прецессия перигелия)
            rel_correction = 1 + 3 * self.G * source["mass"] / (self.c**2 * r)

            # Добавим нелинейность для малых расстояний
            if r < 2e8:
                rel_correction *= 1.5  # Усиление релятивистских эффектов вблизи Земли

            a_mag = self.grav_coeff * self.c**2 * dp_dr / max(p_res, 1e-20) * rel_correction
            a_total += a_mag * direction

        return a_total

    def derivatives(self, t, y):
        names = list(self.system.keys())
        n = len(names)
        dy_dt = np.zeros(4 * n)

        # Обновление позиций
        for i, name in enumerate(names):
            self.system[name]["position"] = y[2*i:2*i+2]
            self.system[name]["velocity"] = y[2*n + 2*i:2*n + 2*i+2]

        # Расчет ускорений с локальным временем
        for i, name in enumerate(names):
            time_factor = self.system[name].get("local_time_factor", 1.0)

            # Производная позиции
            dy_dt[2*i:2*i+2] = self.system[name]["velocity"] * time_factor

            # Производная скорости
            a = self.acceleration(name)
            dy_dt[2*n + 2*i:2*n + 2*i+2] = a * time_factor

        return dy_dt

    def run_simulation(self):
        names = list(self.system.keys())
        n = len(names)

        # Начальный вектор состояния
        y0 = np.zeros(4 * n)
        for i, name in enumerate(names):
            y0[2*i:2*i+2] = self.system[name]["position"]
            y0[2*n + 2*i:2*n + 2*i+2] = self.system[name]["velocity"]

        # Решение системы ОДУ с высокой точностью
        try:
            solution = solve_ivp(
                self.derivatives,
                [0, self.t_max],
                y0,
                t_eval=self.t_points,
                method='DOP853',
                rtol=1e-9,
                atol=1e-9
            )

            if not solution.success:
                raise RuntimeError(f"Решение ОДУ не удалось: {solution.message}")

            # Сохранение траекторий и скоростей
            trajectories = {}
            velocities = {}
            for i, name in enumerate(names):
                trajectories[name] = solution.y[2*i:2*i+2].T
                velocities[name] = solution.y[2*n + 2*i:2*n + 2*i+2].T

            # Относительная траектория Луны
            trajectories["Moon_relative"] = trajectories["Moon"] - trajectories["Earth"]
            velocities["Moon_relative"] = velocities["Moon"] - velocities["Earth"]

            return trajectories, velocities

        except Exception as e:
            print(f"Ошибка симуляции: {e}")
            # Возвращаем статические траектории
            trajectories = {}
            velocities = {}
            for name in names:
                trajectories[name] = np.tile(self.system[name]["position"], (len(self.t_points), 1))
                velocities[name] = np.tile(self.system[name]["velocity"], (len(self.t_points), 1))
            trajectories["Moon_relative"] = trajectories["Moon"] - trajectories["Earth"]
            velocities["Moon_relative"] = velocities["Moon"] - velocities["Earth"]
            return trajectories, velocities

    def plot_results(self, trajectories):
        if "Moon_relative" not in trajectories:
            print("Нет данных для визуализации")
            return

        moon_rel = trajectories["Moon_relative"]

        fig, ax = plt.subplots(figsize=(12, 10))

        # Земля в центре
        ax.plot(0, 0, 'bo', ms=10, label='Земля')

        # Орбита Луны
        ax.plot(moon_rel[:, 0], moon_rel[:, 1], 'r-', alpha=0.5, label='Орбита Луны')

        # Начало и конец
        ax.plot(moon_rel[0, 0], moon_rel[0, 1], 'go', ms=8, label='Начало')
        ax.plot(moon_rel[-1, 0], moon_rel[-1, 1], 'ro', ms=8, label='Конец')

        # Настройки графики
        ax.set_title('Орбита Луны в системе отсчёта Земли\n(БПП иерархическая модель)', fontsize=14)
        ax.set_xlabel('x относительно Земли (м)', fontsize=12)
        ax.set_ylabel('y относительно Земли (м)', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.axis('equal')

        # Автомасштабирование
        if len(moon_rel) > 0:
            moon_range = 1.2 * np.max(np.abs(moon_rel))
            ax.set_xlim(-moon_range, moon_range)
            ax.set_ylim(-moon_range, moon_range)

        plt.tight_layout()
        plt.savefig('bpp_moon_orbit.png', dpi=150)
        plt.show()

        # Создание анимации
        if len(moon_rel) > 10:
            self.create_animation(moon_rel)
        else:
            print("Недостаточно данных для анимации")

    def create_animation(self, moon_rel):
        fig, ax = plt.subplots(figsize=(10, 10))
        if len(moon_rel) > 0:
            range_val = 1.5 * np.max(np.abs(moon_rel))
            ax.set_xlim(-range_val, range_val)
            ax.set_ylim(-range_val, range_val)
        ax.set_title('Движение Луны вокруг Земли в БПП модели')
        ax.set_xlabel('x (м)')
        ax.set_ylabel('y (м)')
        ax.grid(True)

        # Земля
        earth_radius = 1e7
        earth = plt.Circle((0, 0), earth_radius, color='blue', alpha=0.7)
        ax.add_patch(earth)

        # Луна и орбита
        moon_dot, = ax.plot([], [], 'ro', ms=8)
        orbit_line, = ax.plot([], [], 'r-', alpha=0.3)
        time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)

        # Инициализация
        def init():
            moon_dot.set_data([], [])
            orbit_line.set_data([], [])
            time_text.set_text('')
            return moon_dot, orbit_line, time_text

        # Обновление кадра
        def update(frame):
            # Траектория за последние 200 шагов (из-за уменьшения dt)
            start_idx = max(0, frame - 200)
            x = moon_rel[start_idx:frame+1, 0]
            y = moon_rel[start_idx:frame+1, 1]

            orbit_line.set_data(x, y)
            moon_dot.set_data([moon_rel[frame, 0]], [moon_rel[frame, 1]])

            # Время в днях
            days = frame * self.dt / (24 * 3600)
            time_text.set_text(f'День: {days:.2f}')

            return moon_dot, orbit_line, time_text

        # Создание анимации
        ani = FuncAnimation(
            fig, update, frames=len(moon_rel),
            init_func=init, blit=True, interval=30
        )

        # Сохранение
        try:
            ani.save('bpp_moon_animation.mp4', writer='ffmpeg', fps=30, dpi=100)
            print("Анимация сохранена как 'bpp_moon_animation.mp4'")
        except Exception as e:
            print(f"Ошибка сохранения анимации: {e}")
        finally:
            plt.close(fig)

    def calculate_orbital_elements(self, trajectories, velocities):
        """Корректный расчёт орбитальных параметров"""
        moon_rel = trajectories["Moon_relative"]
        moon_vel_rel = velocities["Moon_relative"]

        eccentricity = []
        semi_major_axis = []

        for i in range(len(moon_rel)):
            r_vec = moon_rel[i]
            r = np.linalg.norm(r_vec)
            v_vec = moon_vel_rel[i]
            v = np.linalg.norm(v_vec)

            mu = self.G * self.system["Earth"]["mass"]

            # Угловой момент
            h_vec = np.cross(np.append(r_vec, 0), np.append(v_vec, 0))
            h = np.linalg.norm(h_vec)

            # Вектор эксцентриситета
            e_vec = ((v_vec.dot(v_vec) - mu/r) * r_vec - r_vec.dot(v_vec) * v_vec) / mu
            e = np.linalg.norm(e_vec)

            # Большая полуось через фокальный параметр
            p = h**2 / mu
            a = p / (1 - e**2) if e < 1 else p / (e**2 - 1)

            semi_major_axis.append(a)
            eccentricity.append(e)

        return {
            "semi_major_axis": np.array(semi_major_axis),
            "eccentricity": np.array(eccentricity)
        }

    def plot_solar_influence(self, trajectories, velocities):
        """Улучшенная визуализация влияния Солнца"""
        if "Moon_relative" not in trajectories or "Earth" not in trajectories or "Sun" not in trajectories:
            print("Нет данных для анализа влияния Солнца")
            return

        moon_rel = trajectories["Moon_relative"]
        earth_sun = trajectories["Earth"] - trajectories["Sun"]

        # Угол между направлением на Солнце и Луной
        solar_angles = []
        for i in range(len(moon_rel)):
            try:
                sun_dir = earth_sun[i] / np.linalg.norm(earth_sun[i])
                moon_dir = moon_rel[i] / np.linalg.norm(moon_rel[i])
                angle = np.degrees(np.arccos(np.clip(np.dot(sun_dir, moon_dir), -1.0, 1.0)))
                solar_angles.append(angle)
            except:
                solar_angles.append(0)

        # Радиальное ускорение Луны
        radial_acc = []
        if len(moon_rel) > 2:
            for i in range(1, len(moon_rel)-1):
                r1 = np.linalg.norm(moon_rel[i-1])
                r2 = np.linalg.norm(moon_rel[i+1])
                dt = self.dt * 2
                radial_acc.append((r2 - r1)/dt)

        fig, axs = plt.subplots(2, 1, figsize=(12, 10)) if radial_acc else plt.subplots(1, 1, figsize=(12, 5))

        if radial_acc:
            ax1, ax2 = axs
        else:
            ax1 = axs

        # График угла между Луной и Солнцем
        time_days = np.arange(len(solar_angles)) * self.dt / (24*3600)
        ax1.plot(time_days, solar_angles, 'b-')
        ax1.set_title('Угол между направлениями на Луну и Солнце')
        ax1.set_xlabel('Время (дни)')
        ax1.set_ylabel('Угол (градусы)')
        ax1.grid(True)

        # График радиального ускорения
        if radial_acc:
            time_days_acc = np.arange(len(radial_acc)) * self.dt / (24*3600)
            ax2.plot(time_days_acc, radial_acc, 'r-')
            ax2.set_title('Радиальное ускорение Луны')
            ax2.set_xlabel('Время (дни)')
            ax2.set_ylabel('Ускорение (м/с²)')
            ax2.grid(True)

        plt.tight_layout()
        plt.savefig('solar_influence.png', dpi=150)
        plt.show()

    def run_full_analysis(self):
        trajectories, velocities = self.run_simulation()
        self.plot_results(trajectories)

        # Анализ орбитальных параметров
        orbital_elements = self.calculate_orbital_elements(trajectories, velocities)

        # Визуализация параметров орбиты
        plt.figure(figsize=(12, 8))

        plt.subplot(211)
        plt.plot(orbital_elements["semi_major_axis"])
        plt.title('Большая полуось орбиты Луны')
        plt.ylabel('Метры')
        plt.grid(True)

        plt.subplot(212)
        plt.plot(orbital_elements["eccentricity"])
        plt.title('Эксцентриситет орбиты Луны')
        plt.ylabel('Эксцентриситет')
        plt.xlabel('Шаг симуляции')
        plt.grid(True)

        plt.tight_layout()
        plt.savefig('orbital_parameters.png', dpi=150)
        plt.show()

        # Анализ влияния Солнца
        self.plot_solar_influence(trajectories, velocities)

        return trajectories, velocities, orbital_elements

# Запуск симуляции
simulator = HierarchicalBPPSimulator()
trajectories, velocities = simulator.run_simulation()
simulator.plot_results(trajectories)

# Для полного анализа
trajectories, velocities, orbital_elements = simulator.run_full_analysis()

Анимация сохранена как 'bpp_moon_animation.mp4'
Анимация сохранена как 'bpp_moon_animation.mp4'
